接下来需要解决的问题就是，模型该输出什么样的结果呢？

1. 首先一个最自然的想法就是，既然我们的预测目标是连续值，直接让模型输出连续的预测结果，然后用MSELoss计算损失。

    然而，经过实践发现，主要有两点问题：

    (1). 由于金融数据含有大量的噪声，即使是相同的市场状态，未来的变化也有很大变数，即市场的混沌性。此时如果我们用MSE计算损失，模型为了避免预测错误带来的平方高额损失，会倾向于预测一个比较稳健的平均值来减小损失，导致模型预测的结果围绕在0周围，往往都是非常小的数字；

    (2). 虽然预测值是离散的，但是我们的交易操作却是离散的。如果直接根据预测结果的符号进行交易，再叠加噪声的影响，会很难判断模型的真正预测。比如模型给出的+0.2的预测，这个数字非常小，是否要根据这样的预测进行交易呢？

2. 改为使用MAELoss(L1Loss)，对异常值的惩罚相对温和，让模型更敢于做出偏离0的预测。

    L1 Loss本质是鼓励模型预测中位数，因此，模型输出收缩向0的问题仍然存在。

3. 那么，能否改为分类问题 + BCELoss，将真实值离散为上涨、下跌两个状态，让模型只预测方向呢？

    经过实验，分类预测缓解了前面的问题，但是仍然有两个新问题：

    (1). 真实值离散化之后，丢失了信息，真实值+2和+200是完全不同的结果，模型理应区别对之；

    (2). 在训练过程中，模型需要一些时间来学习上涨形态和下跌形态的特征，但是如果在此之前，logits的差距已经很大，会导致经softmax之后的prob一边倒。这样的情况下，即使模型学会了某个特征，但由于此时prob已经很大或很小，梯度几乎消失，无法反向传播更新参数来传递这个学习到的特征，导致训练失败。

4. 综上所述，我们可以从以下几个方面解决上述问题：

    (1). 混合回归预测和分类预测，用超参数控制损失函数使得模型具有复合能力；

    (2). 从二分类改为三分类：回想人类的交易决策，一个科学的交易策略应当是三分类的，在行情不明确的时候，要有退出市场的选项来规避预期外的风险。行情不明确，无法预测的时候入场，只会增大风险，降低sharpe ratio。因此，我们的模型不应当在任何时候都给出预测，而是应当学会放弃一部分自己无法预测的情况，

    (3). 构建 “软标签”，通过引入波动率变量的形式，对每个样本生成独立的、更加科学的软标签：即使在训练初期模型预测一边倒的情况下，避免模型过拟合风险，同时也能保持梯度流动，增强训练过程的稳定性

模型假设：

1. 每个窗口存在一个hidden return；而realized return只是hidden return的一个样本，我们只能观测到realized return;

2. hidden return distribution 的均值服从n=1时的t分布, 均值为realized return

3. 由于hidden return是不可知的，hidden return落在不同的区间内的概率是不一致的；（例如，假设某日观测到的真实收益是-50，那么隐含收益落在(-inf, 0)区间的概率就显然大于落在(0, +inf)区间的概率）

4. 同期的成交量数据，可以作为隐含波动率的代理，因为价格的波动需要成交量作为驱动，对于大盘尤其是如此。因此当今天大幅放量的时候，即使最终价格变动很小，也不能认为今天的波动率低。（也可以用ATR等振幅指标）

In [1]:
import torch
import torch.nn as nn

class HybridDecoder(nn.Module):
    """
    混合输出解码器架构，对接HybridLoss
    从单一的信息向量中，通过两种方式分别解码出线性预测输出和概率预测输出
    """
    def __init__(self, dim_state, **kwargs):
        super(HybridDecoder, self).__init__(**kwargs)
        self.device = 'cuda:0'
        self.log_prob = nn.Sequential(nn.Linear(dim_state, 3),nn.LogSoftmax(dim = 1))
        self.regress = nn.Linear(dim_state, 1)
    def forward(self, x):
        return torch.concat((self.regress(x), self.log_prob(x)),dim = 1)
    
if __name__ == '__main__':
    state = torch.ones(size = (10, 64))
    hd = HybridDecoder(dim_state = 64)
    print(hd(state).shape)

torch.Size([10, 4])


损失计算：

输入 -> 特征提取层 -> 

-> 分支一（回归）：特征提取层 -> 全连接输出层（1个标量输出） -> Huber Loss

-> 分支二（分类）：特征提取层 -> 全连接输出层（3个 logits 输出） -> KL Div Loss

-> 汇总 Loss = Alpha * Huber Loss  + (1 - Alpha) * KL Div Loss

In [2]:
import torch
import torch.nn as nn

class HybridLoss(nn.Module):
    """
    应用于金融资产收益率预测的损失函数，对接HybridDecoder
    避免离散化为标签时丢失信息，同时利用成交量等波动率参数强化预测效果。
    将线性预测输出输入到Huber损失函数，将概率预测输出输入到KL散度损失函数，再通过超参数控制两个损失的和作为最终损失
    """
    def __init__(self, alpha = 0.5, delta = 50):
        """
        alpha: 损失函数的混合比例，alpha越接近1, Huber 损失占比越大
        delta：Huber损失的内置参数
        """
        super(HybridLoss, self).__init__()
        self.alpha = alpha
        self.huber_loss = nn.HuberLoss(delta = delta, reduction='mean')
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, pred: torch.Tensor, real: torch.Tensor):
        huber_loss = self.huber_loss(pred[:,:1], real[:,:1])
        kl_loss = self.kl_loss(pred[:,1:], real[:,1:])
        return self.alpha * huber_loss + (1-self.alpha)*kl_loss


概率标签的处理：

首先确定一个放弃预测的阈值，真实值绝对值小于阈值时，认为是波动较小、趋势不明显，应当放弃交易，可以参考预测目标的标准差，控制在三种类型的数据比例大致相等。

以真实值作为均值，以成交量（scale之后）作为标准差，计算正态分布下(-inf, -threshold), (-threshold, +threshold), (+threshold, +inf) 三个区间的概率分布；size = (batch_size, 3)，最后concat到原始价格数据旁边size = (batch_size, 4)

(这里不采用t分布的原因是为了简化梯度计算，因为标准差也是成交量估计出来的，本身误差也很大，在这样的误差之下正态分布和t分布的差距可以忽略了)